<img src="https://unidata.ucar.edu/images/logos/badges/badge_unidata_100.jpg" alt="Unidata Logo" style="float: right; height: 98px;">

# Xarray to Scikit-Learn
___

In [1]:
import numpy as np

import xarray as xr
import pandas as pd

## Load in Xarray Tutorial Dataset

In [2]:
ds_ar = xr.tutorial.load_dataset("air_temperature")
ds_ar

<xarray.Dataset>
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time     (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

Future sanity check for size of dataframe later

In [3]:
data_length = np.shape(ds_ar.air.values.ravel())
data_length[0]

3869000

Let's add a second data variable to make the dataset a bit more realistic

In [4]:
ds_ar['air_plus5'] = ds_ar['air']+ 5 
ds_ar

<xarray.Dataset>
Dimensions:    (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat        (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon        (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
  * time       (time) datetime64[ns] 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air        (time, lat, lon) float32 241.2 242.5 243.5 ... 296.5 296.2 295.7
    air_plus5  (time, lat, lon) float32 246.2 247.5 248.5 ... 301.5 301.2 300.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## Make a Pandas Dataframe that is sensible for future analysis in Scikit-learn

Scikit-learn requires a 2D array, with input variables as columns, and each sample as a row. You can do this with xarray stack function

In [5]:
# first need to grab the coordiante names as a tuple

my_tuple = tuple(ds_ar.coords)
desired_values = ('time', 'time1')

In [6]:
reordered_tuple = tuple(value for value in desired_values if value in my_tuple) + tuple(value for value in my_tuple if value not in desired_values)
print(reordered_tuple)

('time', 'lat', 'lon')


In [7]:
df = ds_ar.stack(stacked = reordered_tuple).to_dataframe()
df.head()

air   air_plus5       time   lat    lon
time       lat  lon                                                  
2013-01-01 75.0 200.0  241.199997  246.199997 2013-01-01  75.0  200.0
                202.5  242.500000  247.500000 2013-01-01  75.0  202.5
                205.0  243.500000  248.500000 2013-01-01  75.0  205.0
                207.5  244.000000  249.000000 2013-01-01  75.0  207.5
                210.0  244.099991  249.099991 2013-01-01  75.0  210.0

Sanity check

In [8]:
print('Did we lose any data points?')
df.shape[0] != data_length[0]

Did we lose any data points?


False

Users might want to drop the multi-index, or use it for their analysis. 

### Question:

Is there a smart way using pint/MetPy to put time as the first value in the tuple? 

is it worth the additional overhead within MetPy to simplyfy this code to the user to just do:

xarray_dataset.timestack() 